# Runnable 


LangChain’s most important idea isn’t a specific model—it’s the **Runnable**.

A Runnable is just a tiny, consistent “block” that takes some input and produces some output—with the same controls everywhere:

invoke(input) → run once (sync)

ainvoke(input) → run once (async/await)

batch([inputs]) → run many at once

stream(input) / astream(input) → stream partial results (e.g., tokens)

Think of a Runnable as a Lego piece with standard buttons. You can wrap almost anything—a function, an LLM, a retriever—so it all behaves the same way.


Runnable[Input, Output] is a generic abstract/base in langchain_core.runnables. 
You usually don’t subclass it directly—use helpers like RunnableLambda, 
or just use existing runnables (ChatPromptTemplate, ChatOpenAI, parsers).

## Mental “type” picture

ChatPromptTemplate: Runnable[dict, list[BaseMessage]]

ChatOpenAI: Runnable[list[BaseMessage], AIMessage]

JsonOutputParser[T]: Runnable[str, T]

### invoke method

In [18]:
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")
response = model.invoke("What is capital of UK?  Give me answer in one paragraph, and give like you are teaching to 5 grade students. add one fun fact in the  end")

print(response.content)

The capital of the United Kingdom is London! It is a big and exciting city that is located in England, which is one of the countries in the UK. London is known for its famous landmarks like the Tower of London, Big Ben, and Buckingham Palace, where the Queen lives. It has lots of museums, parks, and even a giant Ferris wheel called the London Eye. People from all over the world visit London to see its sights and learn about its history. Here's a fun fact: London has a secret underground train system called the Tube, and it's one of the oldest in the world!


### batch method

In [14]:
inputs = [
    "What is capital of China",
    "What is capital of spain"
]
responses = model.batch(inputs)
for resp in responses:
    print("-", resp.content)

- The capital of China is Beijing.
- The capital of Spain is Madrid.


### Stream Method

In [19]:
stream_response = model.stream("What is capital of UK?  Give me answer in two paragraph, and give like you are teaching to 5 grade students. add one fun fact in the  end")

for chunk in stream_response:
    # chunk is a ChatGenerationChunk
    print(chunk.content, end="")
print("\n-- end stream --")

The capital of the United Kingdom (UK) is London. London is a very big and busy city located in England. It is not just the political center of the UK but also a place full of history, culture, and fun things to do! In London, you can visit famous landmarks like the Big Ben clock tower, the Tower of London, and Buckingham Palace, where the Queen of England lives. Many people come from all over the world to see these amazing sights and learn more about English history.

London is quite special because it has a mixture of old and new buildings, like ancient castles standing next to modern skyscrapers. It's a city where you can ride on red double-decker buses or take a boat ride on the River Thames. Did you know that London is home to the British Museum? It's one of the largest and most famous museums in the world, where you can see amazing treasures from ancient civilizations! Fun fact: London has a secret underground system called the "Tube," which helps people travel around the city qu

# Custom Runnable

In [17]:
from langchain_core.runnables import RunnableLambda
to_upper = RunnableLambda(lambda s: s.upper())
print(to_upper.invoke("hello")) 

HELLO


In [20]:
# poetry add install  pydantic
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import json, datetime, asyncio

# 1) String preprocessor (trim + collapse spaces)
clean_text = RunnableLambda(lambda s: " ".join(s.strip().split()))
print(clean_text.invoke("  Hello   LangChain \n RunnableLambda  "))   # "Hello LangChain RunnableLambda"
print(clean_text.batch(["  a  ", " b\nc "]))                          # ['a', 'b c']

# 2) Dict enricher (add defaults)
add_defaults = RunnableLambda(lambda d: {**d, "tone": d.get("tone", "friendly")})
print(add_defaults.invoke({"topic": "embeddings"}))
# {'topic': 'embeddings', 'tone': 'friendly'}

# 3) Simple JSON parser (raises on invalid JSON) + retry wrapper
parse_json = RunnableLambda(lambda s: json.loads(s))
robust_parse = parse_json.with_retry()   # retries on exceptions
print(robust_parse.invoke('{"ok":true}'))  # {'ok': True}

# 4) Async variant (add UTC timestamp)
async def add_ts_async(d):
    return {**d, "ts": datetime.datetime.utcnow().isoformat() + "Z"}

add_ts = RunnableLambda(func=lambda d: {**d, "ts": "sync"}, afunc=add_ts_async)
# print(asyncio.run(add_ts.ainvoke({"topic": "runnable"})))
# {'topic': 'runnable', 'ts': '2025-08-27T10:11:12.345678Z'}

# 5) Use RunnableLambda as a tiny pre-processor before a prompt/model call (no LCEL pipes)
prompt = ChatPromptTemplate.from_messages([
    ("system", "Be concise."),
    ("human", "Explain {topic} in one sentence for a {tone} audience.")
])
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prepare inputs → clean/enrich → build messages → invoke model
user_input = {"topic": "vector search"}
prepped = add_defaults.invoke(user_input)# adds tone
print("prepped data", prepped)
messages = prompt.invoke(prepped)                       # prompt is a Runnable
print("messages : ", messages)
reply = llm.invoke(messages)                            # model is a Runnable
print(reply.content)


Hello LangChain RunnableLambda
['a', 'b c']
{'topic': 'embeddings', 'tone': 'friendly'}
{'ok': True}
prepped data {'topic': 'vector search', 'tone': 'friendly'}
messages :  messages=[SystemMessage(content='Be concise.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain vector search in one sentence for a friendly audience.', additional_kwargs={}, response_metadata={})]
Vector search is a method of finding similar items by comparing their numerical representations, or "vectors," in a multi-dimensional space, making it easier to discover related content like images, text, or products.
